# 3D Reconstruction - Cheetah Demo

This notebook illustrates the steps taken to reconstruct 3D points from 2D points estimated by a DeepLabCut (DLC) network.

The required data for this demonstration (DLT coefficients and DLC estimated points) have been provided.

## Extrinsic Calibration

The first step is to determine the relationship between all the cameras in the multi-camera configuration i.e. the extrinsic calibration.  This can be done using __[Argus Wand](http://argus.web.unc.edu/)__ or __[easyWand](http://biomech.web.unc.edu/wand-calibration-tools/)__ (easyWand was used for the 3D cheetah reconstruction) - both these toolboxes produce DLT coefficients as the means to describe the relationship between the cameras, extrinsic camera matrix cannot be used for this particular reconstruction method.

## Reconstruction

The next step is the actual 3D reconstruction using the DLT coefficients determined in the extrinsic calibration and 2D estimates from DLC.  Simply follow the step-by-step instructions below!

### The Reconstruction Function

Below is the function definition that performs the reconstruction. (see __[DLTdv7](http://biomech.web.unc.edu/dltdv/)__)

In [1]:
# define the reconstruction function - adapted from DLTdv7 which was written by Ty Hedrick
import numpy as np
import csv

def dlt_reconstruct(c,camPts):

    # number of frames
    nFrames=len(camPts)
    # number of cameras
    nCams=len(camPts[0])/2

    # setup output variables
    xyz = np.empty((nFrames, 3))
    rmse = np.empty((nFrames, 1))

    # process each frame
    for i in range(1, nFrames+1):
  
        # get a list of cameras with non-NaN [u,v]
        cdx_size = 0
        skip = []
        cdx_temp=np.where(np.isnan(camPts[i-1,0:int(nCams*2)-1:2])==False, 1, 0)
        for a in range(len(cdx_temp)):
            if cdx_temp[a-1] == 1:
                cdx_size = cdx_size + 1
                skip.append(a)
                
        cdx = np.empty((1, cdx_size))
        for b in range(cdx_size):
            cdx[0][b] = b+1        

        # if we have 2+ cameras, begin reconstructing
        if cdx_size>=2:
            # initialize least-square solution matrices
            m1=np.empty((cdx_size*2, 3))
            m2=np.empty((cdx_size*2, 1))
            temp = 0
            
            for z in range(cdx_size*2):
                if z%2==0:
                    m1[z,0]=camPts[i-1,(skip[temp]*2)-2]*c[8,(skip[temp]-1)]-c[0,(skip[temp]-1)]
                    m1[z,1]=camPts[i-1,(skip[temp]*2)-2]*c[9,(skip[temp]-1)]-c[1,(skip[temp]-1)]
                    m1[z,2]=camPts[i-1,(skip[temp]*2)-2]*c[10,(skip[temp]-1)]-c[2,(skip[temp]-1)]
                    m2[z,0]=c[3,skip[temp]-1]-camPts[i-1,(skip[temp]*2)-2]
                    #temp1 = temp1+1
                else:
                    skip[temp]
                    m1[z,0]=camPts[i-1,(skip[temp]*2)-1]*c[8,skip[temp]-1]-c[4,skip[temp]-1]
                    m1[z,1]=camPts[i-1,(skip[temp]*2)-1]*c[9,skip[temp]-1]-c[5,skip[temp]-1]
                    m1[z,2]=camPts[i-1,(skip[temp]*2)-1]*c[10,skip[temp]-1]-c[6,skip[temp]-1]
                    m2[z,0]=c[7, skip[temp]-1]-camPts[i-1,(skip[temp]*2)-1]
                    #temp2 = temp2+1
                    temp+=1
                    
            # get the least squares solution to the reconstruction
            Q, R = np.linalg.qr(m1) # QR decomposition with qr function 
            y = np.dot(Q.T, m2) # Let y=Q'.B using matrix multiplication 
            x = np.linalg.solve(R, y) # Solve Rx=y
            xyz_pts = x.transpose()
            
            xyz[i-1,0:3]=xyz_pts
            
        else:
            xyz[i-1,0:3] = np.NaN
    
    return xyz

Now, set your desired threshold, only the DLC estimates with a likelihood above this threshold will be used in the reconstruction.  A threshold of 0.5 is used in this demo. 

Note: if the camera distortion is severe, it may be ideal to undistort the DLC estimated points prior to reconstruction.

In [2]:
# set threshold
threshold = 0.5

# get names of labels
names = ['r_eye', 'l_eye', 'r_shoulder', 'r_front_knee', 'r_front_ankle', 'r_front_paw', 'spine', 'r_hip', 'r_back_knee', 'r_back_ankle', 'r_back_paw', 'tail1', 'tail2', 'l_shoulder', 'l_front_knee', 'l_front_ankle', 'l_front_paw', 'l_hip', 'l_back_knee', 'l_back_ankle', 'l_back_paw', 'lure', 'tail_base']

# load dlt coefficient
c = np.loadtxt("dltCoefs.csv", delimiter=",")

skiprows = 0
# read in data from DLC
CAM1=np.genfromtxt("CAM1DeepCut_resnet50_CheetahDec3shuffle1_1030000_undistort_fisheye.csv",dtype=float,delimiter=',',skip_header=skiprows)
CAM2=np.genfromtxt("CAM2DeepCut_resnet50_CheetahDec3shuffle1_1030000_undistort_fisheye.csv",dtype=float,delimiter=',',skip_header=skiprows)
CAM3=np.genfromtxt("CAM3DeepCut_resnet50_CheetahDec3shuffle1_1030000_undistort_fisheye.csv",dtype=float,delimiter=',',skip_header=skiprows)
CAM4=np.genfromtxt("CAM4DeepCut_resnet50_CheetahDec3shuffle1_1030000_undistort_fisheye.csv",dtype=float,delimiter=',',skip_header=skiprows)
CAM5=np.genfromtxt("CAM5DeepCut_resnet50_CheetahDec3shuffle1_1030000_undistort_fisheye.csv",dtype=float,delimiter=',',skip_header=skiprows)
CAM6=np.genfromtxt("CAM6DeepCut_resnet50_CheetahDec3shuffle1_1030000_undistort_fisheye.csv",dtype=float,delimiter=',',skip_header=skiprows)
 

### Let the reconstruction begin!

The following will perform the 3D reconstruction for each specified markers and save each set of 3D points as separate .csv files. 

Note: the following has been developed for a 6 camera configuration, if more or less are used please edit accordingly.  However, the dlt_reconstruct() function can be used as is without adjustemnts. 

In [3]:
# thresholding
for x in range(len(names)):
    likelihood_1 = CAM1[:,x*3+2]
    likelihood_2 = CAM2[:,x*3+2]
    likelihood_3 = CAM3[:,x*3+2]
    likelihood_4 = CAM4[:,x*3+2]
    likelihood_5 = CAM5[:,x*3+2]
    likelihood_6 = CAM6[:,x*3+2]
       
    # loops through each frame
    for y in range(len(likelihood_1)):
        indv_like_1 = likelihood_1[y]
        indv_like_2 = likelihood_2[y]
        indv_like_3 = likelihood_3[y]
        indv_like_4 = likelihood_4[y]
        indv_like_5 = likelihood_5[y]
        indv_like_6 = likelihood_6[y]
        
        # if likelihood is below threshold then remove xy coord
        if indv_like_1<=threshold:
            CAM1[y,(x*3):(x*3+2)] = np.NaN
        
        if indv_like_2<=threshold:
            CAM2[y,x*3:x*3+2] = np.NaN
        
        if indv_like_3<=threshold:
            CAM3[y,x*3:x*3+2] = np.NaN
        
        if indv_like_4<=threshold:
            CAM4[y,x*3:x*3+2] = np.NaN
        
        if indv_like_5<=threshold:
            CAM5[y,x*3:x*3+2] = np.NaN
        
        if indv_like_6<=threshold:
            CAM6[y,x*3:x*3+2] = np.NaN
    
    
# loop through each marker for reconstruction
for z in range(len(names)):
    CAM1xy = CAM1[:,z*3:z*3+2]
    CAM2xy = CAM2[:,z*3:z*3+2]
    CAM3xy = CAM3[:,z*3:z*3+2]
    CAM4xy = CAM4[:,z*3:z*3+2]
    CAM5xy = CAM5[:,z*3:z*3+2]
    CAM6xy = CAM6[:,z*3:z*3+2]
    
    # combine
    camPts = np.hstack((CAM1xy, CAM2xy, CAM3xy, CAM4xy, CAM5xy, CAM6xy))
    
    # call the reconstruction function
    xyz = dlt_reconstruct(c, camPts)

    name = (names[z]+"_filtered_0-5.csv")

    # save xyz coords in csv file
    np.savetxt(name, xyz, delimiter = ',', fmt='%1.4f')

## Plot the 3D skeleton

The following will plot the reconstructed 3D points and a 3D cheetah 'skeleton.'  For convenience, only a select number of frames are plotted and not the whole sequece.

Note: the script below has been developed specifically for this task i.e. plotting a cheetah skeleton with our specific markers.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits import mplot3d
import pandas as pd
import matplotlib.animation as animation
import math
import pylab
%matplotlib tk

# create figure for 3D plot
fig = plt.figure()
pylab.rcParams['figure.figsize'] = (15, 5)
plt.tight_layout()
ax = plt.axes(projection='3d')

# names of labels
names = ['r_eye', 'l_eye', 'r_shoulder', 'r_front_knee', 'r_front_ankle', 'r_front_paw', 'spine', 'r_hip', 'r_back_knee', 'r_back_ankle', 'r_back_paw', 'tail_base', 'tail1', 'tail2', 'l_shoulder', 'l_front_knee', 'l_front_ankle', 'l_front_paw', 'l_hip', 'l_back_knee', 'l_back_ankle', 'l_back_paw']

# cmap for plotted points
color = plt.cm.get_cmap('cool', len(names))

def plot3D(k):
    # clear plot
    plt.cla()

    xdata=[]
    ydata=[] 
    zdata=[]

    
    # read in xyz coords for each marker for frame k
    o = 0
    for i in names:
        # 1. change the path to where the 3D .csv files are saved
        xyz_pts = pd.read_csv(i+"_filtered_0-5.csv", skiprows=k, nrows = 1, header=None)
        
        # save xyz coords
        xdata.append(xyz_pts[0][0])
        ydata.append(xyz_pts[1][0])
        zdata.append(xyz_pts[2][0])
    
    
    
    # getting midpoints
    midx = []
    midy = []
    midz = []
    
    # eyes
    midx.append((xdata[0]+xdata[1])/2)
    midy.append((ydata[0]+ydata[1])/2)
    midz.append((zdata[0]+zdata[1])/2)
    
    # shoulders
    midx.append((xdata[2]+xdata[14])/2)
    midy.append((ydata[2]+ydata[14])/2)
    midz.append((zdata[2]+zdata[14])/2)
    
    # hips
    midx.append((xdata[7]+xdata[18])/2)
    midy.append((ydata[7]+ydata[18])/2)
    midz.append((zdata[7]+zdata[18])/2)
    
    
    # initialise array for limbs
    xface = []
    yface = []
    zface = []

    xshoulder = []
    yshoulder = []
    zshoulder = []
    
    xhip = []
    yhip = []
    zhip = []

    xfrontleg_r = []
    yfrontleg_r = []
    zfrontleg_r = []
    
    xfrontleg_l = []
    yfrontleg_l = []
    zfrontleg_l = []
    
    xback = []  # i.e the spine
    yback = []
    zback = []
    
    xbackleg_r = []
    ybackleg_r = []
    zbackleg_r = []
    
    xbackleg_l = []
    ybackleg_l = []
    zbackleg_l = []


    # add points to 'create' limbs
    # connect two eyes
    xface.append(xdata[0]) # right eye
    xface.append(xdata[1]) # left eye
    yface.append(ydata[0])
    yface.append(ydata[1])
    zface.append(zdata[0])
    zface.append(zdata[1])
    
    # shoulders
    xshoulder.append(xdata[2]) # right shoulder 
    xshoulder.append(xdata[14]) # left shoulder
    yshoulder.append(ydata[2])
    yshoulder.append(ydata[14])
    zshoulder.append(zdata[2])
    zshoulder.append(zdata[14])

    # front legs
    xfrontleg_r.append(xdata[2]) # right shoulder
    xfrontleg_r.append(xdata[3])
    xfrontleg_r.append(xdata[4])
    xfrontleg_r.append(xdata[5])
    yfrontleg_r.append(ydata[2])
    yfrontleg_r.append(ydata[3])
    yfrontleg_r.append(ydata[4])
    yfrontleg_r.append(ydata[5])
    zfrontleg_r.append(zdata[2])
    zfrontleg_r.append(zdata[3])
    zfrontleg_r.append(zdata[4])
    zfrontleg_r.append(zdata[5])
    
    xfrontleg_l.append(xdata[14]) # left shoulder
    xfrontleg_l.append(xdata[15])
    xfrontleg_l.append(xdata[16])
    xfrontleg_l.append(xdata[17])
    yfrontleg_l.append(ydata[14])
    yfrontleg_l.append(ydata[15])
    yfrontleg_l.append(ydata[16])
    yfrontleg_l.append(ydata[17])
    zfrontleg_l.append(zdata[14])
    zfrontleg_l.append(zdata[15])
    zfrontleg_l.append(zdata[16])
    zfrontleg_l.append(zdata[17])    
    
    # back/spine
    #xback.append(midx[0])
    xback.append(midx[1])
    xback.append(xdata[6])
    xback.append(midx[2])
    xback.append(xdata[11])
    xback.append(xdata[12])
    xback.append(xdata[13])
    #yback.append(midy[0])
    yback.append(midy[1])
    yback.append(ydata[6])
    yback.append(midy[2])
    yback.append(ydata[11])
    yback.append(ydata[12])
    yback.append(ydata[13])
    #zback.append(midz[0])
    zback.append(midz[1])
    zback.append(zdata[6])
    zback.append(midz[2])
    zback.append(zdata[11])
    zback.append(zdata[12])
    zback.append(zdata[13])
    
    # hips
    xhip.append(xdata[7]) # right hip
    xhip.append(xdata[18]) # left hip
    yhip.append(ydata[7]) 
    yhip.append(ydata[18])
    zhip.append(zdata[7]) 
    zhip.append(zdata[18])
    
    # back legs
    xbackleg_r.append(xdata[7]) # right hip
    xbackleg_r.append(xdata[8])
    xbackleg_r.append(xdata[9])
    xbackleg_r.append(xdata[10])
    ybackleg_r.append(ydata[7])
    ybackleg_r.append(ydata[8])
    ybackleg_r.append(ydata[9])
    ybackleg_r.append(ydata[10])
    zbackleg_r.append(zdata[7])
    zbackleg_r.append(zdata[8])
    zbackleg_r.append(zdata[9])
    zbackleg_r.append(zdata[10])
    
    xbackleg_l.append(xdata[18]) # left hip
    xbackleg_l.append(xdata[19])
    xbackleg_l.append(xdata[20])
    xbackleg_l.append(xdata[21])
    ybackleg_l.append(ydata[18])
    ybackleg_l.append(ydata[19])
    ybackleg_l.append(ydata[20])
    ybackleg_l.append(ydata[21])
    zbackleg_l.append(zdata[18])
    zbackleg_l.append(zdata[19])
    zbackleg_l.append(zdata[20])
    zbackleg_l.append(zdata[21])
 
    # set graph
    ax.set_xlim3d([-1.1, 2.5])
    ax.set_xticklabels([])
    #ax.set_xlabel('X')

    ax.set_ylim3d([-0.2, 1])
    ax.set_yticklabels([])
    #ax.set_ylabel('Y')

    ax.set_zlim3d([-4, -1])
    ax.set_zticklabels([])
    #ax.set_zlabel('Z')
    
    ax.xaxis.grid(False)
    
    # change the view/angle
    ax.view_init(113, 270)

    ax.w_yaxis.set_pane_color((0.3, 0.3, 0.3, 0.3))
    ax.w_xaxis.set_pane_color((0, 0, 0, 0))
    ax.w_zaxis.set_pane_color((0, 0, 0, 0))
    #ax.grid(None)
    
    # plot each point and it's index according to names list
    for i in range(2, len(xdata)): # the eyes are not plotted
        ax.scatter(xdata[i],ydata[i],zdata[i],c=color(i))  # point
       #ax.text(xdata[i],ydata[i],zdata[i],  '%s' % (str(i)), size=10, zorder=1, color='k')  # index
    
    # add the limbs
    ax.plot(xfrontleg_r,yfrontleg_r,zfrontleg_r, color='k')
    ax.plot(xback,yback,zback, color='k')
    ax.plot(xbackleg_r,ybackleg_r,zbackleg_r, color='k')
    #ax.plot(xface, yface, zface, color = 'k')
    ax.plot(xfrontleg_l,yfrontleg_l,zfrontleg_l, color='k')
    ax.plot(xbackleg_l,ybackleg_l,zbackleg_l, color='k')
    ax.plot(xshoulder, yshoulder, zshoulder, color='k')
    ax.plot(xhip, yhip, zhip, color='k')
    
    # wait for press before moving on to next frame
    #plt.waitforbuttonpress()

    
# range of frames
#ani = animation.FuncAnimation(fig, plot3D, range(100, 180))

# selection of frames
ani = animation.FuncAnimation(fig, plot3D, [109, 113, 134, 137, 141])


# to save animation
#FFwriter = animation.FFMpegWriter(fps=3)
#ani.save('14_12_2017LilyFlick1_3Dreconstruction.mp4', writer = FFwriter)

plt.plot()
#ignore error, as figure will pop up!